In [14]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
{Calcination Analysis - Notebook 2
Approximation of calcination experiments with curves and planes.}

{INTERNAL USE ONLY}
"""

__author__ = '{Malte Schade}'
__copyright__ = 'Copyright {2022}, {Calcination Analysis - Notebook 2}'
__version__ = '{1}.{0}.{0}'
__maintainer__ = '{Malte Schade}'
__email__ = '{contact@malteschade.com}'
__status__ = '{PROTOTYPE}'

# built-in modules
import os

# other modules
import numpy as np
import pandas as pd
import scipy.linalg as la
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
import plotly.express as px
import plotly.graph_objects as go

# parameters
PATH_REAC = '/data/Conso/ReactivityData.csv'


#### Approximation function for T/t curve:

$$ f(x) = a-{b^2 \over 1+{x^2 \over 1000c}} $$

In [8]:
def fit(x:float, a:float, b:float, c:float) -> float:
    '''
    Approximation function for scipy curve_fit.

    Parameters
    ----------
    x: float
        Independent variable x.
    a: float
        Function parameter a.
    b: float
        Function parameter b.
    c: float
        Function parameter c.

    Returns
    ----------
    y: float
        Dependant variable y.
    '''
    return a-(b/((x**2/(1000*c))+1))


In [7]:
def get_plane(data: np.ndarray) -> tuple([float, float, float, list, float]):
    '''
    Calculates a quadratic plane of best fit for the datapoints of an experiment set.

    Parameters
    ----------
    data: np.ndarray
        Numpy array with all datapoints for the experiment set.

    Returns
    ----------
    
    '''
    minv = np.min(data, axis=0)
    maxv = np.max(data, axis=0)

    X, Y = np.meshgrid(
        np.arange(minv[0], maxv[0], 10), np.arange(minv[1], maxv[1], 10))
    XX = X.flatten()
    YY = Y.flatten()

    A = np.c_[np.ones(data.shape[0]), data[:, :2], np.prod(
        data[:, :2], axis=1), data[:, :2]**2]
    C, R, _, _ = la.lstsq(A, data[:, 2])
    Z = np.dot(np.c_[np.ones(XX.shape), XX, YY, XX *
               YY, XX**2, YY**2], C).reshape(X.shape)
    return X, Y, Z, C, R


In [9]:
# read reactivity data 
df_raw = pd.read_csv(os.path.dirname(os.getcwd()) + PATH_REAC, encoding="latin_1")
df = df_raw.copy()

# drop data columns with empty data fields
df.dropna(subset=['tseconds', 'temp', 'tcelcius', 'nech'],
          inplace=True)  # nearly 50% dropped??

# drop unnecessaary columns
df.drop(columns=['tdelta', 'tincrease', 'tdelta3',
        'tdelat6', 'fname', 'n_lims', 'e_cd'], inplace=True)

# rename columns for better usability
df.rename(columns={'tseconds': 'ts', 'tcelcius': 'tc'}, inplace=True)

# create multidimensional index
df = df.set_index(['nech', 'temp', 'ts'])
df_raw


/tmp/ipykernel_299453/747262924.py:2: DtypeWarning: Columns (2,5,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(os.path.dirname(os.getcwd()) +


,tseconds,tcelcius,tdelta,tincrease,tdelta3,tdelat6,fname,n_lims,e_cd,nech,temp
0,0.0,20.6,0.0,0.0,0.0,0.0,S1_CFT-4 1060-1200,NaN,CFT 4,CFT004,1060.0
1,2.0,21.0,0.4,10.8,1.0,1.0,S1_CFT-4 1060-1200,NaN,CFT 4,CFT004,1060.0
2,3.0,21.9,1.3,26.0,1.0,1.0,S1_CFT-4 1060-1200,NaN,CFT 4,CFT004,1060.0
3,4.0,23.2,2.6,38.6,1.0,1.0,S1_CFT-4 1060-1200,NaN,CFT 4,CFT004,1060.0
4,5.0,24.5,3.9,46.7,1.0,1.0,S1_CFT-4 1060-1200,NaN,CFT 4,CFT004,1060.0
...,...,...,...,...,...,...,...,...,...,...,...
520454,1412.0,79.8,59.4,2.5,0.1,0.1,S2_SMS 9 1400,NaN,SMS 9,SMS009,1400.0
520455,1413.0,79.8,59.5,2.5,0.1,0.1,S2_SMS 9 1400,NaN,SMS 9,SMS009,1400.0
520456,1414.0,79.8,59.5,2.5,0.1,0.1,S2_SMS 9 1400,NaN,SMS 9,SMS009,1400.0
520457,1415.0,79.7,59.4,2.5,0.0,0.1,S2_SMS 9 1400,NaN,SMS 9,SMS009,1400.0


In [10]:
# create empty result list
result_list = []

# loop through every nech of dataset
for nech in df.reset_index()['nech'].unique():

    # get subsection of dataframe
    df_temp = df.loc[nech].reset_index()

    # llop through all different temperature curves of subsample
    for name, group in df_temp.groupby('temp'):
        points = group['ts']

        # fit user-defined regression curve to data points with bounds and max evolutions
        z, _ = curve_fit(fit, group['ts'], group['tc'], method='trf', maxfev=10000, p0=(
            50, 50, 2000), bounds=([0, 0, 0], [120, 120, 6000]))
        points2 = fit(points, *z)

        # create series from results
        ser_result = pd.Series([nech, name, z[0], z[1], z[2], r2_score(
            group["tc"], points2)], index=['nech', 'temp', 'a', 'b', 'c', 'R2'])

        # append series to result list
        result_list.append(ser_result)

# create result dataframe
df_result = pd.DataFrame(result_list)

# add location column to results by removing numerical characters from nech
df_result['loc'] = df_result['nech'].str.replace('[0-9]', '', regex=True)
df_result


,nech,temp,a,b,c,R2,loc
0,CFT004,700.0,63.294722,30.441941,2262.014108,0.977374,CFT
1,CFT004,900.0,63.288200,26.892731,375.934770,0.913076,CFT
2,CFT004,980.0,64.161830,29.475248,545.915405,0.873489,CFT
3,CFT004,1060.0,58.534286,25.326497,266.389015,0.827275,CFT
4,CFT004,1140.0,63.448626,30.067661,621.239197,0.978582,CFT
...,...,...,...,...,...,...,...
213,SMS017,1300.0,84.970324,48.342630,1254.301075,0.981281,SMS
214,SMS017,1400.0,84.193522,50.058189,3288.797467,0.976301,SMS
215,SMS018,1400.0,83.819562,46.997005,1261.899987,0.977911,SMS
216,SMS019,1400.0,86.697888,50.328251,3146.239652,0.976029,SMS


In [11]:
df.reset_index()['nech'].unique()


array(['CFT004', 'MBNA003', 'NEL031', 'NEL032', 'NEL033', 'NEL034',
       'NEL035', 'NBF018', 'TX001', 'TX002', 'AGF003', 'AGF002', 'AGF001',
       'AGF004', 'AGF005', 'AGF006', 'AGF007', 'CLM108', 'SPZ011',
       'SPZ016', 'SPZ022', 'SPZ012', 'SPZ013', 'SPZ015', 'SPZ017',
       'NGG133', 'NGG134', 'RE609', 'RE612', 'RE614', 'RE616', 'RE617',
       'RE618', 'STB017', 'GB257', 'GB256', 'GB258', 'GB263', 'GB264',
       'GB564', 'JE382', 'JE389', 'JE483', 'JE484', 'JE485', 'JE487',
       'JE488', 'JE489', 'JE490', 'JE491', 'JE492', 'JE493', 'JE494',
       'JE495', 'JE496', 'JE497', 'JE498', 'JE499', 'JE500', 'JE501',
       'KYV000', 'BK092', 'BK117', 'BK119', 'BK122', 'BK118', 'BK121',
       'IST004', 'SNB153', 'SNB154', 'SNB155', 'SNB156', 'SNB159',
       'SNB161', 'OMN079', 'RUS478', 'RUS477', 'RUS479', 'RUS891', '000',
       '920', '924', '928', '932', 'RUS943', 'RUS944', 'RUS946', 'SMS010',
       'SMS011', 'SMS015', 'SMS017', 'SMS018', 'SMS019', 'SMS009'],
      dtype=obj

In [13]:
# select one sample and only use every 10th datapoint for visualization
df_test = df.loc['NGG134'].reset_index().iloc[::10]

# create graph objects figure
fig = go.Figure()

# add raw datapoints as scatter markers 
fig.add_trace(go.Scatter3d(x=df_test['temp'], y=df_test['ts'], z=df_test['tc'],
                           mode='markers', marker={'size': 3}, opacity=0.5))

# calculate regression lines for every temperature curve
for name, group in df_test.groupby('temp'):

    # fit user-defined regression curve to data points with bounds and max evolutions
    points = group['ts']
    z, _ = curve_fit(fit, group['ts'], group['tc'], method='trf', maxfev=10000, p0=(
        50, 50, 2000), bounds=([0, 0, 0], [120, 120, 6000]))
    points2 = fit(points, *z)

    # fit quadratic regression plane to data points
    X, Y, Z, C, R = get_plane(df_test.values)

    # plot regression plane
    fig.add_trace(go.Surface(x=X, y=Y, z=Z, opacity=0.3,
                  showlegend=False, showscale=False))

    # plot regression curve
    fig.add_trace(go.Scatter3d(x=[name]*len(points), y=points, z=points2,
                               mode='lines', line={'width': 5, 'color': 'red'}))

    # print regression parameters
    print(nech, name)
    print(f'Coefficients:\na={z[0]}\nb={z[1]}\nc={z[2]}')
    print(f'R2:\n{r2_score(group["tc"],points2)}\n')

# show figure
fig.show()

SMS009 980.0
Coefficients:
a=77.58999972512773
b=38.04490149167961
c=1.0315864251664406
R2:
0.996790232217311

SMS009 1060.0
Coefficients:
a=76.5837475434065
b=35.9085965193636
c=1.4464254736921867
R2:
0.9958994729682026

SMS009 1140.0
Coefficients:
a=79.07792909550241
b=37.92481691277597
c=7.731755230265802
R2:
0.9933877158250973

SMS009 1200.0
Coefficients:
a=78.5046709440332
b=35.234930095786744
c=37.786252452468695
R2:
0.9976707208468221

[-3.20791727e+02  7.60781203e-01  2.08676554e-01 -3.09104450e-05
 -3.85880343e-04 -2.07907231e-04] 1384.8535430413544


In [11]:
# show 3d scatter plot with a, b, c parameters and location as color
px.scatter_3d(df_result[df_result['R2'] > 0.90], x='a', y='b',
              z='c', color='loc', hover_data=['nech', 'temp', 'R2'])


In [20]:
# show 3d scatter plot with a, b, c parameters and R2 as color
px.scatter_3d(df_result[df_result['R2'] > 0.90], x='a', y='b',
              z='c', color='R2', hover_data=['nech', 'temp', 'R2'])
